In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load pre-trained model
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   8%|7         | 83.9M/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Function to recommend similar products based on the clicked product name
def recommend_similar_products(clicked_product_name, product_data, top_k=5):
    # Convert the clicked product name to lowercase
    clicked_product_name = clicked_product_name.lower()

    # Get the embedding representation for the clicked product name
    clicked_product_embedding = model.encode([clicked_product_name], convert_to_tensor=True)

    # Convert all product names in the data to lowercase
    product_names = [name.lower() for name in product_data['name'].tolist()]

    # Get the embedding representations for all product names
    product_embeddings = model.encode(product_names, convert_to_tensor=True)

    # Calculate similarity between the clicked product name and all other product names
    similarity_scores = util.pytorch_cos_sim(clicked_product_embedding, product_embeddings)[0]

    # Sort products based on similarity score
    similar_products_indices = similarity_scores.argsort(descending=True)

    # Debugging: Display similarity scores and similar products
    print("Similarity Scores:", similarity_scores)

    # Get similar products based on embedding similarity
    similar_products_by_embedding = [product_names[i] for i in similar_products_indices[:top_k]]

    # Debugging: Display similar products based on embedding
    print("Similar Products by Embedding:", similar_products_by_embedding)

    # Find product names that contain the searched substring
    similar_products_by_substring = [name for name in product_names if clicked_product_name in name]

    # Debugging: Display similar products based on substring
    print("Similar Products by Substring:", similar_products_by_substring)

    # Combine results from both approaches without duplicates
    combined_results = list(dict.fromkeys(similar_products_by_substring + similar_products_by_embedding))

    # Limit results to top_k
    final_recommendations = combined_results[:top_k]

    return final_recommendations


In [ ]:
# Demo:
if __name__ == "__main__":
    # Baca dataset produk
    df = pd.read_csv('product.csv', delimiter=';')

    # Tampilkan nama kolom untuk memastikan 'name' ada
    print("Kolom yang tersedia:", df.columns)

    # Klik pada sebuah produk (misalnya, 'floor')
    clicked_product_name = 'cof'

    # Get Similar Recommendation
    recommendations = recommend_similar_products(clicked_product_name, df, top_k=10)

    # Display Product Recommendation
    print("Rekomendasi produk yang mirip dengan", clicked_product_name, ":")
    for product in recommendations:
        print(product)

Kolom yang tersedia: Index(['id', 'name', 'description', 'price_range', 'min_order', 'order_req',
       'supply_ability', 'history_view_product', 'user_id', 'order_click',
       'image'],
      dtype='object')
Similarity Scores: tensor([0.4437, 0.2559, 0.4534, 0.3590, 0.3204, 0.4177, 0.3357, 0.4437, 0.2747,
        0.4641, 0.2705, 0.2517, 0.3420, 0.2685, 0.4500, 0.3227, 0.4903, 0.3420,
        0.2394, 0.3595, 0.3276, 0.3490, 0.3919, 0.4091, 0.2994, 0.4427, 0.4177,
        0.4599, 0.4437, 0.3124, 0.4404, 0.4417, 0.4112, 0.4690, 0.4437, 0.3163,
        0.2685, 0.4046, 0.3920, 0.3880, 0.4091, 0.3083, 0.4437, 0.4408, 0.3880,
        0.2212, 0.2394, 0.4046, 0.3919, 0.4599, 0.3083, 0.4457, 0.2991, 0.2584,
        0.3880, 0.3179, 0.4186, 0.3183, 0.4404, 0.3475, 0.2705, 0.3183, 0.4437,
        0.2685, 0.4046, 0.3124, 0.4112, 0.4641, 0.4457, 0.4599, 0.3045, 0.4232,
        0.4966, 0.2447, 0.3204, 0.4232, 0.2935, 0.4641, 0.2991, 0.4966, 0.3880,
        0.2367, 0.4404, 0.4500, 0.4417, 0.4417, 0